In [1]:
from neo4j import GraphDatabase
import dotenv
import os
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
load_status = dotenv.load_dotenv("C:\\Users\\Usuario\\Documents\\Workspace\\Estudio-Psiquiatricos\\code\\neo4j\\Neo4j-921e6a7b-Created-2025-10-13.txt")

if load_status is False:
    raise RuntimeError('Environment variables not loaded.')

URI = os.getenv("NEO4J_URI")
AUTH = (os.getenv("NEO4J_USERNAME"), os.getenv("NEO4J_PASSWORD"))


driver = GraphDatabase.driver(URI, auth=AUTH)

DIAG_LABEL = "Diagnostico"   
DIAG_NAME_PROP = "terminoEN"     
PACIENTE_LABEL = "Paciente"      

In [3]:
def run_query(query, params=None):
    with driver.session() as session:
        result = session.run(query, params or {})
        return [record.data() for record in result]

In [4]:
# 1) ¿Hay nodos Paciente?
q_count_pat = f"MATCH (p:{PACIENTE_LABEL}) RETURN count(p) AS cnt"
cnt_pat = run_query(q_count_pat)[0]["cnt"]
print(f"Pacientes encontrados: {cnt_pat}")

# 2) Cargar nodos Diagnostico (id -> nombre)
q_nodos = f"MATCH (d:{DIAG_LABEL}) RETURN id(d) AS id, d.{DIAG_NAME_PROP} AS nombre"
nodos_raw = run_query(q_nodos)
id2name = {row["id"]: row["nombre"] or f"diag_{row['id']}" for row in nodos_raw}
print(f"Diagnósticos cargados: {len(id2name)}")

Pacientes encontrados: 1567


Received notification from DBMS server: <GqlStatusObject gql_status='01N01', status_description='warn: feature deprecated with replacement. id is deprecated. It is replaced by elementId or consider using an application-generated id.', position=<SummaryInputPosition line=1, column=30, offset=29>, raw_classification='DEPRECATION', classification=<NotificationClassification.DEPRECATION: 'DEPRECATION'>, raw_severity='WARNING', severity=<NotificationSeverity.WARNING: 'WARNING'>, diagnostic_record={'_classification': 'DEPRECATION', '_severity': 'WARNING', '_position': {'offset': 29, 'line': 1, 'column': 30}, 'OPERATION': '', 'OPERATION_CODE': '0', 'CURRENT_SCHEMA': '/'}> for query: 'MATCH (d:Diagnostico) RETURN id(d) AS id, d.terminoEN AS nombre'


Diagnósticos cargados: 1746


In [5]:

# 3) Construir grafo de co-ocurrencia según disponibilidad de Pacientes
G = nx.Graph()

# Añadir nodos al grafo con etiqueta/nombre
for nid, nombre in id2name.items():
    G.add_node(nid, label=nombre)

if cnt_pat > 0:
    # Extraer, para cada paciente, la lista de diagnósticos conectados (cualquier tipo de relación)
    # Usamos cualquier relación entrante/saliente entre paciente y diagnostico
    q_rels = f"""
    MATCH (p:{PACIENTE_LABEL})-[r]->(d:{DIAG_LABEL})
    RETURN id(p) AS pid, collect(DISTINCT id(d)) AS diagnosticos
    """
    rels = run_query(q_rels)
    # Construir co-ocurrencia: dos diagnósticos conectados si aparecen en el mismo paciente
    for row in rels:
        diags = row["diagnosticos"] or []
        # si el paciente solo tiene 0 o 1 diagnóstico, no genera aristas
        for i in range(len(diags)):
            for j in range(i+1, len(diags)):
                a, b = diags[i], diags[j]
                if G.has_edge(a, b):
                    G[a][b]["weight"] += 1
                else:
                    G.add_edge(a, b, weight=1)
    print("Grafo de co-ocurrencia construido a partir de pacientes.")
else:
    # Fallback: extraer relaciones directas entre diagnósticos
    # Capturamos cualquier relación de Diagnostico -> Diagnostico (p. ej. DIAGNOSTICO_ASOCIADO, DIAGNOSTICOS_PSIQUIATRICO)
    q_diag_links = f"""
    MATCH (d1:{DIAG_LABEL})-[r]->(d2:{DIAG_LABEL})
    RETURN id(d1) AS a, id(d2) AS b, type(r) AS relType, count(*) AS cnt
    """
    links = run_query(q_diag_links)
    for row in links:
        a, b, cnt = row["a"], row["b"], row["cnt"]
        # Tratamos la proyección como no dirigida (si quieres dirigir, cambia a DiGraph)
        if G.has_edge(a, b):
            G[a][b]["weight"] += cnt
        else:
            G.add_edge(a, b, weight=cnt)
    print("Grafo construido a partir de relaciones directas diagnóstico→diagnóstico.")

Received notification from DBMS server: <GqlStatusObject gql_status='01N01', status_description='warn: feature deprecated with replacement. id is deprecated. It is replaced by elementId or consider using an application-generated id.', position=<SummaryInputPosition line=3, column=12, offset=56>, raw_classification='DEPRECATION', classification=<NotificationClassification.DEPRECATION: 'DEPRECATION'>, raw_severity='WARNING', severity=<NotificationSeverity.WARNING: 'WARNING'>, diagnostic_record={'_classification': 'DEPRECATION', '_severity': 'WARNING', '_position': {'offset': 56, 'line': 3, 'column': 12}, 'OPERATION': '', 'OPERATION_CODE': '0', 'CURRENT_SCHEMA': '/'}> for query: '\n    MATCH (p:Paciente)-[r]->(d:Diagnostico)\n    RETURN id(p) AS pid, collect(DISTINCT id(d)) AS diagnosticos\n    '
Received notification from DBMS server: <GqlStatusObject gql_status='01N01', status_description='warn: feature deprecated with replacement. id is deprecated. It is replaced by elementId or consid

Grafo de co-ocurrencia construido a partir de pacientes.


In [6]:
# 4) Normalizar / preparar pesos para algoritmos que los interpretan como 'coste'
# NetworkX interpreta 'weight' como la longitud de la arista (coste) para shortest-path.
# Como nuestras aristas con mayor 'weight' significan mayor co-ocurrencia (fuerza), para betweenness
# es preferible invertirlas: coste = 1/weight. Si weight==0 (no debería), evitamos división por 0.
for u, v, data in G.edges(data=True):
    w = data.get("weight", 1)
    data["inv_weight"] = 1.0 / w if w and w > 0 else float("inf")

print(f"Nodos en G: {G.number_of_nodes()}  Aristas en G: {G.number_of_edges()}")

Nodos en G: 1746  Aristas en G: 40657


In [7]:
# 5) Cálculo de métricas
print("Calculando Degree centrality...")
deg = nx.degree_centrality(G)  # normalizada por defecto (0..1)

print("Calculando Betweenness centrality (usa inv_weight para costes)...")
# weighted betweenness: usamos 'inv_weight' como coste para shortest paths
bet = nx.betweenness_centrality(G, weight='inv_weight', normalized=True)

print("Calculando Eigenvector centrality...")
# eigenvector puede fallar en grafos grandes; probamos con numpy y cayendo a método iterativo si hace falta
try:
    eig = nx.eigenvector_centrality_numpy(G, weight='weight')
except Exception as e:
    print("eigenvector_centrality_numpy falló:", e, "Intentando power method con más iteraciones...")
    try:
        eig = nx.eigenvector_centrality(G, max_iter=1000, tol=1e-06, weight='weight')
    except Exception as e2:
        print("También falló eigenvector (posible grafo demasiado grande o mal condicionado):", e2)
        #Fallback: set zeros
        eig = {n: 0.0 for n in G.nodes()}

Calculando Degree centrality...
Calculando Betweenness centrality (usa inv_weight para costes)...
Calculando Eigenvector centrality...
eigenvector_centrality_numpy falló: `eigenvector_centrality_numpy` does not give consistent results for disconnected graphs Intentando power method con más iteraciones...


In [8]:
# 6) Crear DataFrame con resultados
rows = []
for node in G.nodes():
    rows.append({
        "node_id": node,
        "diagnostico": G.nodes[node].get("label", id2name.get(node, f"diag_{node}")),
        "degree": deg.get(node, 0.0),
        "betweenness": bet.get(node, 0.0),
        "eigenvector": eig.get(node, 0.0)
    })

df_metrics = pd.DataFrame(rows)
# ordenar por cada métrica y guardarlo
df_metrics.sort_values("betweenness", ascending=False).head(20)

# Guardar resultados
df_metrics.to_csv("diagnosticos_centralidades.csv", index=False)
print("Resultados guardados en diagnosticos_centralidades.csv")

Resultados guardados en diagnosticos_centralidades.csv


In [9]:
# 7) (Opcional) Mostrar top-N por cada métrica
N = 15
print("\nTop por Betweenness:")
print(df_metrics.sort_values("betweenness", ascending=False).head(N)[["diagnostico","betweenness","degree","eigenvector"]])

print("\nTop por Degree:")
print(df_metrics.sort_values("degree", ascending=False).head(N)[["diagnostico","degree","betweenness","eigenvector"]])

print("\nTop por Eigenvector:")
print(df_metrics.sort_values("eigenvector", ascending=False).head(N)[["diagnostico","eigenvector","degree","betweenness"]])

# Cerrar driver



Top por Betweenness:
                                            diagnostico  betweenness  \
370                        Otros tipos de esquizofrenia     0.595755   
371                      Esquizofrenia, no especificada     0.192701   
363                                       Esquizofrenia     0.132870   
364                             Esquizofrenia paranoide     0.125523   
1552  Problemas relacionados con la vida en una inst...     0.065664   
1701  Incumplimiento del paciente con otro tratamien...     0.063142   
570                    Hipertensión esencial (primaria)     0.061585   
373                               Trastornos delirantes     0.050863   
1606  Historia familiar de otros trastornos mentales...     0.034285   
1516  Contacto para reconocimiento médico psiquiátri...     0.026728   
379   Psicosis no especificada no debida a sustancia...     0.021345   
350   Dependencia de nicotina no especificada, sin c...     0.020692   
243         Diabetes mellitus tipo 2 sin c

In [10]:
def get_edges():
    query = """
    MATCH (d1:Diagnostico)-[:DIAGNOSTICO_ASOCIADO|:DIAGNOSTICO_PSIQUIATRICO]-(d2:Diagnostico)
    RETURN d1.nombre AS source, d2.nombre AS target
    """
    with driver.session() as session:
        result = session.run(query)
        return pd.DataFrame(result.data())

edges_df = get_edges()
edges_df.head()

Received notification from DBMS server: <GqlStatusObject gql_status='01N01', status_description="warn: feature deprecated with replacement. ':DIAGNOSTICO_ASOCIADO|:DIAGNOSTICO_PSIQUIATRICO' is deprecated. It is replaced by ':DIAGNOSTICO_ASOCIADO|DIAGNOSTICO_PSIQUIATRICO'.", position=<SummaryInputPosition line=2, column=50, offset=50>, raw_classification='DEPRECATION', classification=<NotificationClassification.DEPRECATION: 'DEPRECATION'>, raw_severity='WARNING', severity=<NotificationSeverity.WARNING: 'WARNING'>, diagnostic_record={'_classification': 'DEPRECATION', '_severity': 'WARNING', '_position': {'offset': 50, 'line': 2, 'column': 50}, 'OPERATION': '', 'OPERATION_CODE': '0', 'CURRENT_SCHEMA': '/'}> for query: '\n    MATCH (d1:Diagnostico)-[:DIAGNOSTICO_ASOCIADO|:DIAGNOSTICO_PSIQUIATRICO]-(d2:Diagnostico)\n    RETURN d1.nombre AS source, d2.nombre AS target\n    '
Received notification from DBMS server: <GqlStatusObject gql_status='01N52', status_description='warn: property key do

""


In [13]:

from pyvis.network import Network
from IPython.display import IFrame

# Usar el grafo G y las métricas ya calculadas: bet, deg, eig

net = Network(height="750px", width="100%", bgcolor="#ffffff", font_color="black")

for node in G.nodes():
    label = G.nodes[node].get("label", str(node))
    net.add_node(
        node,
        label=label,
        title=f"<b>{label}</b><br>Betweenness: {bet.get(node,0):.3f}<br>Degree: {deg.get(node,0):.3f}<br>Eigenvector: {eig.get(node,0):.3f}",
        value=deg.get(node,0)*1000,  # tamaño proporcional al degree
        color="red" if bet.get(node,0) > 0.05 else "lightblue"  # nodos puente en rojo
    )

# Añadir enlaces
for source, target in G.edges():
    net.add_edge(source, target, color="gray")

# Mostrar en el notebook o exportar a HTML
net.show("grafo_diagnosticos.html")
display(IFrame("grafo_diagnosticos.html", width="100%", height="750px"))


grafo_diagnosticos.html


AttributeError: 'NoneType' object has no attribute 'render'

In [ ]:
driver.close()